##**Solar energy feed-in data in 15-minute intervals**

In this notebook we read and process the data downloaded as a csv file from the website below. The data relate to the TenneT area of Schleswig-Holstein and are available both historically and forecast.

Website: https://netztransparenz.tennet.eu/electricity-market/transparency-pages/transparency-germany/network-figures/actual-and-forecast-solar-energy-feed-in/schleswig-holstein/

As a first step, download the data for 2020 to 2024 from the specified website and store it as "sh_solar_feedin_2020_24.csv" in your google drive.

In [1]:
# packages
import os
import pandas as pd
from datetime import datetime

In [2]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# searching for files and load data

def search_file(directory, filename):
    for root, dirs, files in os.walk(directory):
        if filename in files:
            return os.path.join(root, filename)
    return None

search_directory = '/content/drive/My Drive'
file_name = 'sh_solar_feedin_2020_24.csv' # change the name if required (if stored differently)
file_path = search_file(search_directory, file_name)

df_solar_feedin = pd.read_csv(file_path, sep = ';')

In [10]:
# rename/translate column names
column_name_mapping = {
    'Date': 'date',
    'Start Time': 'start',
    'End Time' : 'end',
    'Forecast in MW': 'forecast_solar_MW',
    'Actual in MW': 'actual_solar_MW'
}

df_solar_feedin = df_solar_feedin.rename(columns=column_name_mapping)

# convert time columns to datetime and set start date as new index
df_solar_feedin['start'] = pd.to_datetime(df_solar_feedin['date'] + ' ' + df_solar_feedin['start'])
df_solar_feedin = df_solar_feedin.drop('date', axis = 1)
df_solar_feedin = df_solar_feedin.drop('end', axis = 1)
df_solar_feedin.set_index('start', inplace=True)

# change german to english numbering conventions
columns_to_float = ['forecast_solar_MW', 'actual_solar_MW']

for column in columns_to_float:
  df_solar_feedin[column] = df_solar_feedin[column].str.replace(',', '.')
  df_solar_feedin[column] = df_solar_feedin[column].astype(float)

df_solar_feedin = df_solar_feedin.div(15).round(2)

In [12]:
# create folder in google drive if not exists
folder_path = '/content/drive/My Drive/wind_curtailment_prediction'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created successfully.")
else:
    print("Folder already exists.")

Folder created successfully.


In [14]:
# save csv
df_solar_feedin.to_csv('/content/drive/My Drive/wind_curtailment_prediction/solar_feedin_2020_24.csv',sep = ';', index_label='timestamp')